In [ ]:
"""PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion
where{
        ?palabras po:descripcion_palabra ?descripcion.
        ?palabras ^po:pi_tiene_palabra ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        FILTER (
            REGEX(str(?descripcion), 'gestor','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
"""

In [52]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/InvestigacionesUdenar/sparql")
sparql.setQuery("""
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion
where{
        ?palabras po:descripcion_palabra ?descripcion.
        ?palabras ^po:pi_tiene_palabra ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        FILTER (
            REGEX(str(?descripcion), 'gestor')||
            REGEX(str(?descripcion), 'conocimiento')
        )
    }
""")

#order by DESC(?c)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()
results = results['results']['bindings']

In [53]:
id_docs = [result['id_pi']['value'] for result in results ]
len(id_docs)

341

In [55]:
from collections import Counter 
from itertools import repeat, chain 
    
# printing initial ini_list 
#print ("initial list", str(id_docs), "Cantidad:" ,len(id_docs)) 
  
# sorting on bais of frequency of elements 
result = list(chain(i for i, c in Counter(id_docs).most_common())) 
  
# printing final result 
print("final list", str(result), "Cantidad:",len(result)) 

final list ['204', '193', '230', '231', '312', '320', '352', '375', '433', '13', '119', '154', '229', '422', '443', '1', '12', '18', '38', '47', '56', '71', '203', '218', '226', '233', '243', '245', '281', '410', '419', '451', '3', '4', '6', '7', '8', '9', '10', '14', '21', '24', '25', '28', '30', '34', '39', '48', '49', '53', '55', '58', '62', '63', '64', '65', '68', '73', '75', '77', '80', '83', '91', '94', '98', '103', '106', '107', '111', '117', '122', '126', '127', '131', '143', '157', '159', '160', '163', '168', '170', '177', '180', '188', '190', '191', '196', '198', '201', '202', '205', '212', '214', '219', '220', '223', '224', '232', '238', '247', '248', '251', '261', '263', '266', '271', '272', '276', '280', '282', '284', '286', '288', '291', '292', '293', '304', '305', '308', '310', '311', '321', '329', '331', '332', '334', '336', '337', '338', '343', '345', '346', '349', '350', '360', '368', '371', '376', '377', '380', '382', '394', '405', '407', '409', '416', '417', '430', 

In [2]:
results = sparql.query().convert()


In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Scripts')
from ontologia import *

In [2]:
sync_reasoner() g

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/sangeeky/anaconda3/envs/tesis/lib/python3.7/site-packages/owlready2/hermit:/home/sangeeky/anaconda3/envs/tesis/lib/python3.7/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpuvntmp6v
* Owlready2 * HermiT took 14.36101770401001 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [2]:
palabra = (ontologia.Palabra.instances())[0]
trabajo = (palabra.palabra_describe_pi)[0]
trabajo

OntologiaInvestigacionPrueba.pi1

In [3]:
print(trabajo.is_a)
#print(ontologia.get_instances_of(parent))
print(ontologia.get_parents_of(Facultad))
print(ontologia.get_children_of(Universidad))

[OntologiaInvestigacionPrueba.Proyecto_investigacion]
[OntologiaInvestigacionPrueba.Universidad]
[OntologiaInvestigacionPrueba.VIIS, OntologiaInvestigacionPrueba.Facultad]


In [2]:
graph = default_world.as_rdflib_graph()

In [2]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
""")


In [11]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:palabra_describe_pi ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabras po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
""")

In [5]:
#Directa
trabajos = list(graph.query("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
"""))

#trabajos
                

In [6]:
trabajos[1]

(rdflib.term.Literal('204', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),
 rdflib.term.Literal('HONGOS FITOPATÓGENOS ASOCIADOS AL LAUREL DE CERA Morella pubescens', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),
 rdflib.term.URIRef('http://www.semanticweb.org/OntologiaInvestigacionPrueba#pi204'),
 rdflib.term.Literal('3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))

In [3]:
#Inversa
trabajos = graph.query("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:descripcion_palabra ?descripcion.
        ?palabras po:palabra_describe_pi ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
""")

trabajos

http://www.semanticweb.org/OntologiaInvestigacionPrueba#d.i.a . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#d.i.a . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#e.e . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#e.e . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#d.c . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#d.c . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#s.a . does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/OntologiaInvestigacionPrueba#s.a . does not look lik